In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import json
from query import query_events_db
from nostr_sdk import Keys, Client, EventBuilder, Filter, PublicKey, EventId, Options
from datetime import timedelta
import time

from sqlalchemy import create_engine, Column, BigInteger, String
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.orm import declarative_base, sessionmaker



In [ ]:
engine = create_engine(
    "postgresql://postgres@localhost:5432/postgres"
)

In [ ]:
keys = Keys.generate()
# print(keys.public_key().to_bech32())
opts = Options().timeout(timedelta(seconds=1000))
client = Client.with_opts(keys, opts)
# client = Client(keys)
client.add_relay("wss://relay.damus.io")
client.add_relay("wss://nostr.oxtr.dev")
client.add_relay("wss://nostr.openchain.fr")
client.connect()
# PublicKey.from_hex('031ea306188fee631a86f1a0e49cee8f3d62592b2e7be5f8eb1788e327db340a').to_bech32()

In [ ]:
filter = Filter().limit(100000)
reqevents = client.req_events_of([filter], None)

In [ ]:
pablo = "npub1l2vyh47mk2p0qlsku7hg0vn29faehy9hy34ygaclpn66ukqp3afqutajft"
pk = PublicKey.from_bech32(pablo)
filter = Filter().author(pk.to_hex()).limit(50000)

events_pablo = client.get_events_of([filter], timedelta(seconds=1000))
df_pablo = pd.DataFrame([json.loads(event.as_json()) for event in events_pablo]).set_index("id")

In [ ]:
def get_events(npub):
    # Subtract 7 days from the current date and time
    start_ts = (datetime.now() - timedelta(days=7)).timestamp()
    # timestamp = int(seven_days_ago.timestamp())
    pk = PublicKey.from_bech32(npub)
    filter = Filter().author(pk.to_hex()).since(start_ts)
    event = client.get_events_of([filter], timedelta(seconds=1000))

In [ ]:
df_pablo = pd.read_pickle('pablo.pkl')

In [ ]:
# pd.to_pickle(df_pablo, 'pablo.pkl')
len(df_pablo)

In [ ]:
temp = df_pablo.loc['5ff739f0d1998ed71a52e45d043a1a98d95aa4cac4072d7046cc01df448b5227'].iloc[0]['tags'][0]

In [ ]:
temp[0], len(temp)

In [ ]:
lst

In [ ]:
# Extract rows based on condition and create new DataFrames
e_rows = []
p_rows = []

for index, row in df_pablo.iterrows():
    for lst in row['tags']:
        if lst and lst[0] == 'e':
            if len(lst) == 4: 
                # ["e", "1fcc...", "wss://nos.lol/", "reply"]
                e_rows.append([row.name, lst[1], lst[2], lst[3]])
            elif len(lst) == 3:
                # ['e', '10c9a19..', 'wss://nostr.wine/']
                # ['e', '9ee62e263f', '']
                e_rows.append([row.name, lst[1], lst[2], None])
            elif len(lst) == 2:
                # ["e", "1fcc...""]
                e_rows.append([row.name, lst[1], None, None])
            else:
                raise(ValueError, "expected 2-4 fields for e tags")
        elif lst and lst[0] == 'p':
            if len(lst) == 4:
                # ['p', '97c70a...', 'wss://relayable.org', 'hodlbod']
                # ['p', 'fa984b..', '', 'mention' ]
                p_rows.append([row.name, lst[1], lst[2], lst[3]])
            elif len(lst) == 3:
                # ['p', 'fa984bd...', 'pablof7z']
                # ['p', 'b708f73...', 'wss://nostr-relay.wlvs.space']
                p_rows.append([row.name, lst[1], lst[2], None])
            elif len(lst) == 2:
                # ['p', 'ee11a5']
                p_rows.append([row.name, lst[1], None, None])
            else:
                raise(ValueError, "expected 2-4 fields for p tags")

# reply table
df_e = pd.DataFrame(e_rows, columns=['source_id', 'event_id', 'relay_url', 'marker']).set_index(["source_id", "event_id"])
# mention table
df_p = pd.DataFrame(p_rows, columns=['source_id', 'event_id', 'relay_url', 'marker']).set_index(["source_id", "event_id"])

# remove empty fields
# if len(lst[2]) == 0:  # ["e", "1fcc...", ""]
#     e_rows.append([row.name, lst[1], None, None])


In [ ]:
# df.to_sql('events', engine, if_exists="replace", dtype={
#     'tags': JSONB,
#     'tags_relay_url': JSONB,
# })

df_e.to_sql('replys', engine, if_exists="replace")

In [ ]:
df_p.to_sql('mentions', engine, if_exists="replace")

In [ ]:
results = pd.read_sql("select * from replys", engine)

In [ ]:
df_e.head()

In [ ]:
results

In [ ]:
df_e['e_col4'].unique()

In [ ]:
df_p['p_col3'].unique()

In [ ]:
df_pablo.head()

In [ ]:
tags_count = []
for item in df_pablo['tags']:
    print(item)
    break
# result = [(lst[0], len(lst)) if lst else (None, 0) for lst in df_pablo['tags']]

# print(result)

In [ ]:
result[0]

In [ ]:
df_pablo.iloc[1]['tags']

In [ ]:
# def query_events(client, kind=None, num_limit=1000):
filter = Filter().limit(50000)
events = client.get_events_of([filter], timedelta(seconds=1000))
df = pd.DataFrame([json.loads(event.as_json()) for event in events]).set_index("id")

In [ ]:
out = df['tags'].apply(lambda x: x.keys() if x else None)

In [ ]:
df.tags.iloc[0]

In [ ]:
from collections import defaultdict
all_tags = defaultdict(int)
for row in out.values:
    if row is None:
        continue
    for item in list(row):
        all_tags[item] += 1

In [ ]:
all_tags

In [ ]:
len(all_tags)

In [ ]:
df.iloc[0]['pubkey']

In [ ]:
# Using the `size` method which returns the count of each group
grouped = df.groupby('pubkey').size().reset_index(name='count')

# OR using the `count` method
# grouped = df.groupby('kind')['kind'].count().reset_index(name='count')


In [ ]:
grouped.sort_values('count', ascending=False).head(10)

In [ ]:
# pd.to_pickle(df, 'events_50000.pkl')
# df = pd.read_pickle('events_100.pkl')

In [ ]:
PublicKey.from_hex(df_pablo.pubkey.unique()[0]).to_bech32()

In [ ]:
for i in range(20):
    print(df_pablo.iloc[i]['tags'])

In [ ]:
df

In [ ]:
df_pablo.iloc[9]['tags']

In [ ]:
df_pablo.iloc[13]['tags']

In [ ]:
df_pablo.head()

In [ ]:
df_pablo.to_sql('pablo', engine, if_exists="replace", dtype={'tags': JSONB})

In [ ]:
# df = query_events(client)
# loses association with key (e or p may have different relay url)
df["tags_relay_url"] = df["tags"].apply(
    lambda x: {item[0]: item[2] for item in x if len(item) > 2} or None
)
df['tags'] = df['tags'].apply(lambda x: {item[0]: item[1] if len(item) > 1 else None for item in x} or None)

df.to_sql('events', engine, if_exists="replace", dtype={
    'tags': JSONB,
    'tags_relay_url': JSONB,
})

# sql_query = "SELECT * FROM events"
# sql_query = "SELECT * FROM events WHERE tags ? 't'"
# sql_query = "SELECT * FROM events;"

# df_results = pd.read_sql(sql_query, engine)


In [ ]:
# Create a SQLAlchemy engine and session factory
Session = sessionmaker(bind=engine)

# Define the ORM class for the 'events' table
Base = declarative_base()

class Events(Base):
    __tablename__ = "events"
    id = Column(String, primary_key=True)
    created_at = Column(BigInteger)
    kind = Column(BigInteger)
    tags = Column(JSONB)
    tags_relay_url = Column(JSONB)
    pubkey = Column(String)
    content = Column(String)
    sig = Column(String)


In [ ]:
# Create a session and query the table using the ORM
with Session() as session:
    results = session.query(Events).all()

# Convert the results into a pandas DataFrame
df = pd.DataFrame([r.__dict__ for r in results])

# Removing unnecessary meta-data related columns
# df = df.drop('_sa_instance_state', axis=1)

print(df)

In [ ]:
npub = "npub1dergggklka99wwrs92yz8wdjs952h2ux2ha2ed598ngwu9w7a6fsh9xzpc"
pk = PublicKey.from_bech32(npub)
pk.to_hex()

In [ ]:
event_id = EventId.from_hex(df.loc[0, 'id'])
event_id.to_bech32()

In [ ]:
event_id.to_bech32()

In [ ]:
type(keys), type(keys.public_key()), type(keys.public_key().to_bech32())

In [ ]:
# sql_query = "SELECT * FROM events"
# sql_query = "SELECT * FROM events WHERE tags ? 't'"
sql_query = "SELECT * FROM events WHERE tags->>'t' = 'sm41623576'";

df_results = pd.read_sql(sql_query, engine)

In [ ]:
# create kind name -> kind val dict
# handle 3rd field (recommended relay url optional) in tag list (see nips1)
# get events for all kinds
# get zaps
# large query

In [ ]:
def query_zaps():
    filter = Filter().kind(9735).limit(10)
    events = client.get_events_of([filter], timedelta(seconds=10))
    return pd.DataFrame([json.loads(event.as_json()) for event in events])